In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import LSTM,Dropout,GRU,MaxPooling1D,Flatten,concatenate
from keras.models import Sequential
from keras.layers import Dense,Concatenate,Bidirectional
from keras.layers.convolutional import Conv1D
import tensorflow as tf
from keras.utils.vis_utils import model_to_dot
from keras.layers import RepeatVector
from keras.layers import TimeDistributed,Activation
from sklearn.preprocessing import MinMaxScaler
from keras import Input,Model

FD1_train = pd.read_csv("train_FD001.txt",delimiter=" ",header = None)
FD1_test = pd.read_csv("test_FD001.txt",delimiter=" ",header = None)
RUL_file=pd.read_csv("RUL_FD001.txt",header = None)
##L' unica differenza con la maggior parte dei prerpcessing degli altri articoli e che non vengono effettuati drop delle feature perchè il processo di feature engineering è svolto dall' autoencoder
FD1_test=FD1_test.drop([26, 27], axis = 1)
columns = {0:'unit_number',1:'time_in_cycles',2:'opSetting1',3:'opSetting2',4:'opSetting3',5:'sensor1',6:'sensor2',
           7:'sensor3',8:'sensor4',9:'sensor5',10:'sensor6',11:'sensor7',12:'sensor8',13:'sensor9',14:'sensor10',
           15:'sensor11',16:'sensor12',17:'sensor13',18:'sensor14',19:'sensor15',20:'sensor16',
           21:'sensor17',22:'sensor18',23:'sensor19',24:'sensor20',25:'sensor21'}
FD1_test.columns=columns
FD1_test = FD1_test.rename(columns=columns)
#FD1_test=FD1_test.drop(columns = ['opSetting1','opSetting2','opSetting3','sensor1','sensor5','sensor6','sensor10','sensor16','sensor18','sensor19','sensor17'], axis =1)



FD1_train=FD1_train.drop([26, 27], axis = 1)
columns = {0:'unit_number',1:'time_in_cycles',2:'opSetting1',3:'opSetting2',4:'opSetting3',5:'sensor1',6:'sensor2',
           7:'sensor3',8:'sensor4',9:'sensor5',10:'sensor6',11:'sensor7',12:'sensor8',13:'sensor9',14:'sensor10',
           15:'sensor11',16:'sensor12',17:'sensor13',18:'sensor14',19:'sensor15',20:'sensor16',
           21:'sensor17',22:'sensor18',23:'sensor19',24:'sensor20',25:'sensor21'}
FD1_train = FD1_train.rename(columns=columns)
#FD1_train=FD1_train.drop(columns = ['opSetting1','opSetting2','opSetting3','sensor1','sensor5','sensor6','sensor10','sensor16','sensor18','sensor19','sensor17'], axis =1)

# calcolo RUL
def Calcolo_RUL(df): 
    df_copy=df.copy()
    gruppo=df_copy.groupby('unit_number')['time_in_cycles'].max().reset_index()
    gruppo.columns = ['unit_number','max_cycles_fu']
    df_copy = df_copy.merge(gruppo, on=['unit_number'], how='left')
    df['RUL'] = df_copy['max_cycles_fu'] - df_copy['time_in_cycles']
    return df
Calcolo_RUL(FD1_train)
#normalizzazione train test
cols_normalize = FD1_train.columns.difference( #non considero unit_number,time_in_cycles,RUL per normalizzazione
    ['unit_number', 'time_in_cycles', 'RUL'])
  

min_max_scaler = MinMaxScaler()

norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(FD1_train[cols_normalize]),
                             columns=cols_normalize,
                             index=FD1_train.index)

join_df = FD1_train[FD1_train.columns.difference(cols_normalize)].join(norm_train_df)
FD1_train = join_df.reindex(columns=FD1_train.columns)

cols_normalizetest = FD1_test.columns.difference(  #non considero unit_number,time_in_cyclesper normalizzazione
    ['unit_number', 'time_in_cycles'])  

norm_train_df_test = pd.DataFrame(min_max_scaler.transform(FD1_test[cols_normalizetest]),
                             columns=cols_normalizetest,
                             index=FD1_test.index)

join_df = FD1_test[FD1_test.columns.difference(cols_normalizetest)].join(norm_train_df_test)
FD1_test = join_df.reindex(columns=FD1_test.columns)
FD1_test=FD1_test.drop(columns = ['time_in_cycles'], axis =1)
data_cols= FD1_test.columns
##
## reshape dati test
def prepare_test_data(df):
    data_list = []
    for unit_number in df.unit_number.unique():
        unit = df[df.unit_number == unit_number]
        data_list.append(np.array(unit[data_cols])[-31:, :])
    return np.stack(data_list)
FD1_test=prepare_test_data(FD1_test)
FD1_test=FD1_test[:,:,1:]


## upper bound per RUL
rul_clip_limit = 120
FD1_train=FD1_train.clip(upper=rul_clip_limit)

FD1_c=FD1_train.copy()
FD1_train = FD1_train.drop(['RUL','time_in_cycles'], axis=1)
## reshape dati di train
def gen_sequence(id_df, seq_length, seq_cols):
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

seq_gen = (list(gen_sequence(FD1_train[FD1_train['unit_number']==unit_number],31, FD1_train.columns))
           for unit_number in FD1_train['unit_number'].unique())

seq_array = np.concatenate(list(seq_gen)).astype(np.float32)


seq_array=seq_array[:,:,1:] #elimino unit_number dal train
print(seq_array.shape) # dimensione dei dati di train

def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

label_gen = [gen_labels(FD1_c[FD1_c['unit_number']==id], 31, ['RUL'])
             for id in FD1_c['unit_number'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)




a=Input(shape=(31,24))
encoded=(LSTM(12,return_sequences=True))(a)
decoded=(LSTM(24,return_sequences=True))(encoded)
autoencoder = Model(a, decoded)
encoder = Model(a, encoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(seq_array,seq_array,batch_size=50,epochs=50,validation_split=0.10,verbose=1)
en=encoder.predict(seq_array)
FD1_test=encoder.predict(FD1_test)
from numpy import var


model= Sequential()
model.add(Bidirectional(LSTM(100,input_shape=(31,12), activation='tanh', return_sequences=True)))
model.add(Bidirectional(LSTM(50,input_shape=(31,12), activation='tanh',return_sequences=False)))
model.add(Flatten())
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='RMSprop', loss='mse',metrics = [tf.keras.metrics.RootMeanSquaredError()])
model.fit(en,label_array ,batch_size=200,epochs=32,verbose=1)
y_pred= model.predict(FD1_test)
from sklearn.metrics import mean_squared_error
from math import sqrt
RUL_file=pd.read_csv("RUL_FD001.txt",header = None)
print(sqrt(mean_squared_error(RUL_file, y_pred)))
print(y_pred)
print((mean_squared_error(RUL_file, y_pred)))


(17531, 31, 24)
Epoch 1/50
316/316 [==============================] - 5s 8ms/step - loss: 1.5465 - val_loss: 0.6920
Epoch 2/50
316/316 [==============================] - 2s 5ms/step - loss: 0.6953 - val_loss: 0.6731
Epoch 3/50
316/316 [==============================] - 2s 5ms/step - loss: 0.6778 - val_loss: 0.6606
Epoch 4/50
316/316 [==============================] - 2s 5ms/step - loss: 0.6672 - val_loss: 0.6555
Epoch 5/50
316/316 [==============================] - 2s 6ms/step - loss: 0.6628 - val_loss: 0.6527
Epoch 6/50
316/316 [==============================] - 2s 5ms/step - loss: 0.6590 - val_loss: 0.6504
Epoch 7/50
316/316 [==============================] - 2s 5ms/step - loss: 0.6587 - val_loss: 0.6494
Epoch 8/50
316/316 [==============================] - 2s 6ms/step - loss: 0.6567 - val_loss: 0.6484
Epoch 9/50
316/316 [==============================] - 2s 5ms/step - loss: 0.6573 - val_loss: 0.6475
Epoch 10/50
316/316 [==============================] - 2s 5ms/step - loss: 0.6551 - 